## This code apply MetalTiger to produce normalized text

In [ ]:
import os
src_dir = 'all_text'
tar_dir = 'all_speech'
i=0
count = 1
with os.scandir(src_dir) as files:
    
    for file in files:
        
        nam= str(file)
        file=nam[11:-2]
        nam='speech-form-'+ nam[21:-2]
        
        file= 'all_text/'+file
        #print(file)
        #print(nam)
        output_file =  'all_speech/'+nam
        
        if i < count:
            
            print('started: ',nam)

            #! ./MetalTiger transform -config metaltiger-master/sample_models/lm_clean/en_US/spokenform.toml -input $file \
            #        >$output_file
        

        print('Done upto', nam, i)
        i +=1
    print('Complete!')

### This code apply metal tiger's text_cleaning.toml for text cleaning  

In [ ]:
import os
src_dir = 'all_text'
i=0
count = -1
with os.scandir(src_dir) as files:
    
    for file in files:
        
        nam= str(file)
        file=nam[11:-2]
        
        
        
        #print(file)
        #print(nam)
        output_file =  'all_text_cleaned/'+file
        file = 'all_text/'+file
        if i > count:
            
            print('started: ',file)

           # ! ./MetalTiger transform -config metaltiger-master/sample_models/lm_clean/en_US/text_cleaning.toml -input $file \
           #         >$output_file
        

        print('Done upto', file, i)
        i +=1
    print('Complete!')

#### Applying sclite to produce comparsion

In [22]:
import os
src_dir = 'all_text_cleaned'
i=0
count = -1
with os.scandir(src_dir) as files:
    
    for file in files:
        
        nam= str(file)
        file=nam[21:-2]

        ref =  'all_text_cleaned/text-form-'+file
        hyp = 'all_speech/speech-form-'+file
        tar = 'Comparison-files/comp-'+file
        if i > count:
            
            
            !cat $ref | tr '\n' ' ' > single-test-out/ref.txt
            !echo "(gar)" >> single-test-out/ref.txt
            !cat $hyp | tr '\n' ' ' > single-test-out/hyp.txt
            !echo "(gar)" >> single-test-out/hyp.txt

            !./sclite -i swb -r single-test-out/ref.txt -h single-test-out/hyp.txt -o pralign sum -l 140 -O single-test-out
            !cp single-test-out/hyp.txt.pra $tar
            
            
        
        
        print('Done upto', file, i)
        i +=1
    print('Complete!')

sclite: 2.10 TK Version 1.3
Begin alignment of Ref File: 'single-test-out/ref.txt' and Hyp File: 'single-test-out/hyp.txt'
Error: extract_speaker can't locate SWB id (gar)
    Alignment# 1 for speaker           

    Writing scoring report to 'single-test-out/hyp.txt.sys'
    Writing string alignments to 'single-test-out/hyp.txt.pra'

Successful Completion
Done upto VS-1332-LX-2021-810359.txt 0
sclite: 2.10 TK Version 1.3
Begin alignment of Ref File: 'single-test-out/ref.txt' and Hyp File: 'single-test-out/hyp.txt'
Error: extract_speaker can't locate SWB id (gar)
    Alignment# 1 for speaker           

    Writing scoring report to 'single-test-out/hyp.txt.sys'
    Writing string alignments to 'single-test-out/hyp.txt.pra'

Successful Completion
Done upto VS-648-ESQMIA-J7160935.txt 1
sclite: 2.10 TK Version 1.3
Begin alignment of Ref File: 'single-test-out/ref.txt' and Hyp File: 'single-test-out/hyp.txt'
Error: extract_speaker can't locate SWB id (gar)
    Alignment# 1 for speaker    

In [ ]:
ref = 'all_text_cleaned/text-form-VS-849-ESQMIA-J7147578.txt'
hyp = 'all_speech/speech-form-VS-849-ESQMIA-J7147578.txt'

!cat $ref | tr '\n' ' ' > single-test-out/ref.txt
!echo "(gar)" >> single-test-out/ref.txt
!cat $hyp | tr '\n' ' ' > single-test-out/hyp.txt
!echo "(gar)" >> single-test-out/hyp.txt

!./sclite -i swb -r single-test-out/ref.txt -h single-test-out/hyp.txt -o pralign sum -l 140 -O single-test-out

## Preparing final data from comparison files V1.0

In [53]:
def findText(line,startpos,l):
    
    t=''
    words = line[startpos:].split(' ')
    cle_words= []
    
    i=0
    
    for word in words:
        #if i < 15:
            if word != '':
                cle_words.append(word)
           # i +=1
       # else:
          #  break

    for i in range(l):
        t =t+ cle_words[i]+ ' '
        
    #print(t)
    return t


In [45]:



def dataExtractor(file):
    
    #step-1 make a single string of speech,text and eval from a whole text file
    import re
    with open(file,'r') as f:
        lines=f.readlines()

    linecount =0

    for line in lines:
        linecount +=1

    print(linecount)
    SpeechLine = ''
    TextLine = ''
    EvalLine = ''

    i=16          #avoid first 16 lines

    while True:

        if (i+3) < linecount: 

            TextLine += lines[i][8:]
            SpeechLine += lines[i+1][8:]
            EvalLine += lines[i+2][8:]

            i +=4
        else: break   
    SpeechLine = re.sub('\n','', SpeechLine)
    TextLine = re.sub('\n','',TextLine)
    EvalLine = re.sub('\n','',EvalLine)






    #step-2 find location of 'I',"S" inside evalLIne

    template = []

    for i in range(len(EvalLine)):
        if EvalLine[i] == 'I' or EvalLine[i] == 'S':
            template.append(i)



    #step-3: start with a 'S' find next 'S' and check if it is the next word of sppechline. if yes then find next else it's the end 



    textData = []
    speechData = []

    i=0

    while True:

        if EvalLine[template[i]] == 'S':

            startpos_text = template[i]
            len_text= 1

            curr_s = i
            b= i+1

            while True:
                if b > (len(template)-1):
                    break
                elif EvalLine[template[b]] == 'S':

                    next_s= b

                    #slice_from_curr_s
                    slices_from_curr = SpeechLine[template[curr_s]:].split(' ')
                    slices_from_next =  SpeechLine[template[next_s]:].split(' ')


                    if slices_from_curr[1]== slices_from_next[0] :
                        curr_s = b
                        len_text +=1
                        b +=1
                        continue
                    else:
                        break
                else:
                    break


            a=i-1
            while True:
                if a < 0:
                    break

                elif EvalLine[template[a]] == 'S':
                    break


                elif EvalLine[template[a]] == 'I':
                    a= a-1
                    continue

            startpos_speech = template[(a+1)]

            len_speech = b-a-1

            t= findText(TextLine,startpos_text,len_text)
            s= findText(SpeechLine,startpos_speech, len_speech)

            textData.append(t)
            speechData.append(s)    
          
            i = b

        else:
            i +=1

        if i > (len(template)-1):
            break

   
    return textData, speechData


In [ ]:
file = 'Comparison-files/comp-VS-1035-ESQMIA-J7374393.txt'

import os
parenText = []
parenSpeech = []
dir= '../formatter/Comparison-files'



with os.scandir(dir) as entries:
    for entry in entries:
        
        text, speech = dataExtractor(entry)
        parenText += text
        parenSpeech += speech
        
        
import pandas as pd
df = pd.DataFrame()
df['Text-form']= parenText
df['Speech-form']= parenSpeech      
        

In [86]:
#df.to_csv('FinalData.csv')
for i in range(1,2,1):
    print(i)

1


## Preparing final data from comparison files V 2.0

Algorithm: Declare two list one for storing text form and other for speech form. we will go forward by comparing two word lines if we see match then we add them in both list. If we find mismatch that means it's the start of the normalization and will store the words in different variable untill we get match words again. Once we get matched word we are end of the normalized words. We will now add the words which was unmatched as a single sample to text form and speech form.

In [57]:



def LinesExtractor(file):
    
    #step-1 make a single string of speech,text and eval from a whole text file
    import re
    with open(file,'r') as f:
        lines=f.readlines()

    linecount =0

    for line in lines:
        linecount +=1

        
    SpeechLine = ''
    TextLine = ''
    EvalLine = ''

    i=16          #avoid first 16 lines

    while True:

        if (i+3) < linecount: 

            TextLine += lines[i][8:]
            SpeechLine += lines[i+1][8:]
            EvalLine += lines[i+2][8:]

            i +=4
        else: break   
    SpeechLine = re.sub('\n','', SpeechLine)
    TextLine = re.sub('\n','',TextLine)
    EvalLine = re.sub('\n','',EvalLine)
    
    
    return TextLine, SpeechLine, EvalLine

In [113]:
def finalizer(t,s):   #prepares text and speech coloumn from text and speech line

    speech_form = []
    text_form = []

    text_words=  t.split(' ')
    temp = []
    for word in text_words:
        if word != '':
            temp.append(word)

    text_words= temp 

    speech_words= s.split(' ')
    temp = []
    for word in speech_words:
        if word != '':
            temp.append(word)
    speech_words= temp

    is_match = 1


    text_sample= ''
    speech_sample= ''

    for i in range(len(text_words)):

        if text_words[i] == speech_words[i]:

            if is_match == 0:

                speech_form.append(speech_sample)
                text_form.append(text_sample)

            is_match = 1
            text_sample =''   #reinitialize text_sample and speech_sample after one update
            speech_sample= ''
            speech_form.append(text_words[i])
            text_form.append(text_words[i])

        else:
            is_match = 0 
            speech_sample  += ' '+speech_words[i]
            if re.match('\*',text_words[i]):
                continue
            text_sample    += ' '+ text_words[i]



 

    return text_form,speech_form


In [121]:

import os
import pandas as pd
df = pd.DataFrame()

parenText = []
parenSpeech = []
dir= '../formatter/Comparison-files'

with os.scandir(dir) as entries:
    for entry in entries:
        
        avoid = str(entry)[11:-2]
       
        if avoid == '.ipynb_checkpoints':
            continue
       
        t,s,e = LinesExtractor(entry)
        text, speech= finalizer(t,s)
        if len(text) != len(speech):
            print(len(text),len(speech))
            print(avoid)
            continue
        
        parenText += text
        parenSpeech += speech
        
for i in range(len(parenText)):
    parenText[i] = parenText[i].lower()
    parenSpeech[i]= parenSpeech[i].lower()

df['Text-form']= parenText
df['Speech-form']= parenSpeech      
        



In [127]:
df.to_csv('FinalDatasetWithContext.csv',index= False)

In [120]:
a= 'hello BRO'
print(a.lower())

hello bro


In [112]:

text_words=  t.split(' ')
temp = []
for word in text_words:
    if word != '':
        temp.append(word)

text_words= temp 

speech_words= s.split(' ')
temp = []
for word in speech_words:
    if word != '':
        temp.append(word)
speech_words= temp


for i in range(len(speech_words)):
    if text_words[i] != speech_words[i]:
        print(text_words[i],'--', speech_words[i])
        print('at: ',i)

INC -- INCORPORATED
at:  55
***** -- FIFTH
at:  75
*** -- TWO
at:  76
******** -- THOUSAND
at:  77
5TH -- TWENTY
at:  78
2021 -- ONE
at:  79
5 -- FIVE
at:  141
INC -- INCORPORATED
at:  196
6 -- SIX
at:  294
**** -- NINE
at:  427
***** -- EIGHT
at:  428
*** -- ONE
at:  429
**** -- ZERO
at:  430
9810A -- A
at:  431
**** -- NINE
at:  441
***** -- EIGHT
at:  442
*** -- ONE
at:  443
**** -- ZERO
at:  444
9810A -- A
at:  445
7 -- SEVEN
at:  472
8 -- EIGHT
at:  662
**** -- NINE
at:  727
***** -- EIGHT
at:  728
*** -- ONE
at:  729
**** -- ZERO
at:  730
9810A -- A
at:  731
**** -- NINE
at:  817
***** -- EIGHT
at:  818
*** -- ONE
at:  819
**** -- ZERO
at:  820
9810A -- A
at:  821
**** -- NINE
at:  840
***** -- EIGHT
at:  841
*** -- ONE
at:  842
**** -- ZERO
at:  843
9810A -- A
at:  844
9 -- NINE
at:  849
*** -- TWO
at:  879
******** -- THOUSAND
at:  880
2007 -- SEVEN
at:  881
6 -- SIX
at:  883
***** -- NINTH
at:  936
*** -- TWO
at:  937
9TH -- THOUSAND
at:  938
2018 -- EIGHTEEN
at:  939
***** --

In [109]:
print(text[2730:3000])
print(speech[2730:3000])

['farm', 'paid', 'based', 'on', 'a', ' 1A1B', 'which', 'is', 'a', 'a', 'bid', 'to', 'ultimately', 'a', 'bid', 'to', 'repair', 'estimate', 'approved', 'by', 'us', 'is', 'there', 'anything', 'in', 'this', 'letter', 'that', 'says', 'our', 'payment', 'was', 'based', 'upon', 'a', 'estimate', 'approved', 'by', 'us', 'it', 'does', 'say', 'our', 'program', 'pricing', 'which', 'is', 'the', 'program', 'price', 'repair', 'estimate', 'so', 'state', "farm's", 'program', 'pricing', "let's", 'talk', 'about', 'that', 'for', 'a', 'moment', 'what', 'program', 'are', 'we', 'talking', 'about', 'the', 'offer', 'and', 'acceptance', 'program', 'well', 'which', 'i', 'the', 'offer', 'and', 'acceptance', 'program', 'is', 'ultimately', 'the', 'state', 'farm', 'glass', 'program', 'so', 'in', 'the', 'last', 'sentence', 'of', 'the', ' 2ND', 'paragraph', 'says', 'our', 'program', 'pricing', 'came', 'in', 'at', ' $ 488 POINT 38', 'when', 'it', 'says', 'our', 'program', 'pricing', 'is', 'it', 'referring', 'to', 'the',

In [111]:
a= t.split(' ')
b= s.split(' ')

for i in range(len(a)):
    if a[i]!= b[i]:
        print(a[i],'---',b[i])

INC --- INCORPORATED
 --- a
 --- slash
 --- a
 --- slash
 --- o
 --- shea
 --- pennington
 --- v
 --- state
a --- 
slash --- farm
a --- mutual
slash --- automobile
o --- insurance
shea --- company
pennington --- today's
v --- date
state --- is
 --- october
farm --- the
mutual --- FIFTH
automobile --- TWO
insurance --- THOUSAND
company --- TWENTY
today's --- ONE
date --- 
is --- before
october --- going
the --- on
***** --- the
*** --- record
******** --- 
5TH --- the
 --- witness
 --- positively
 --- identified
2021 --- himself
before --- to
going --- me
on --- as
the --- mark
record --- hansen
 --- via
the --- a
witness --- florida
positively --- driver's
identified --- license
himself --- my
to --- apologies
me --- counsel
as --- will
mark --- you
hansen --- 
via --- please
a --- state
florida --- your
driver's --- appearance
license --- for
my --- the
apologies --- record
counsel --- your
will --- firm
you --- who
 --- you
please --- represent
state --- and
your --- that
appearance 

IndexError: list index out of range

## IOB tagger data preparation

In [87]:
def findTag(SpeechLine,tagline,startpos_speech,len_speech):
    prev_words= SpeechLine[:startpos_speech].split(' ')
 
    n= len(prev_words)
    tagline[n]='B'
    for i in range(1,len_speech,1):
        tagline[(n+i)] = 'I'        
    
    return tagline

In [89]:
file = 'Comparison-files/comp-VS-1035-ESQMIA-J7374393.txt'

#step-1 make a single string of speech,text and eval from a whole text file
import re
with open(file,'r') as f:
    lines=f.readlines()

linecount =0

for line in lines:
    linecount +=1

print(linecount)
SpeechLine = ''
TextLine = ''
EvalLine = ''

i=16          #avoid first 16 lines

while True:

    if (i+3) < linecount: 

        TextLine += lines[i][8:]
        SpeechLine += lines[i+1][8:]
        EvalLine += lines[i+2][8:]

        i +=4
    else: break   
SpeechLine = re.sub('\n','', SpeechLine)
TextLine = re.sub('\n','',TextLine)
EvalLine = re.sub('\n','',EvalLine)






#step-2 find location of 'I',"S" inside evalLIne

template = []

for i in range(len(EvalLine)):
    if EvalLine[i] == 'I' or EvalLine[i] == 'S':
        template.append(i)



#step-3: start with a 'S' find next 'S' and check if it is the next word of sppechline. if yes then find next else it's the end 



#textData = []
#speechData = []



words= SpeechLine.split(' ')
tagline= []
for word in words:
    tagline.append('O')


i=0

while True:

    if EvalLine[template[i]] == 'S':

        startpos_text = template[i]
        len_text= 1

        curr_s = i
        b= i+1

        while True:
            if b > (len(template)-1):
                break
            elif EvalLine[template[b]] == 'S':

                next_s= b

                #slice_from_curr_s
                slices_from_curr = SpeechLine[template[curr_s]:].split(' ')
                slices_from_next =  SpeechLine[template[next_s]:].split(' ')


                if slices_from_curr[1]== slices_from_next[0] :
                    curr_s = b
                    len_text +=1
                    b +=1
                    continue
                else:
                    break
            else:
                break


        a=i-1
        while True:
            if a < 0:
                break

            elif EvalLine[template[a]] == 'S':
                break


            elif EvalLine[template[a]] == 'I':
                a= a-1
                continue

        startpos_speech = template[(a+1)]

        len_speech = b-a-1

        #t= findText(TextLine,startpos_text,len_text)
        #s= findText(SpeechLine,startpos_speech, len_speech)

        tagline = findTag(SpeechLine,tagline,startpos_speech,len_speech)
        #textData.append(t)
        #speechData.append(s)    

        i = b

    else:
        i +=1

    if i > (len(template)-1):
        break

2685


In [90]:
print(tagline)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B',

In [91]:
print(SpeechLine)

 to spanish and spanish to english in thank you my name is julio fernandez a florida state notary public and court reporter and  this deposition is being held videoconference equipment the witness and reporter are not in the same room the witness will be  sworn in remotely pursuant to agreement of all parties the parties stipulate that the testimony is being given as if the witness  was sworn in person does everyone agree yes yes thank you ms aguilar can you please lift up your right hand to be sworn margot  aguilar having been duly sworn through the english-spanish all questions and answers are through the interpreter unless otherwise  noted direct examination ma'am would you state your whole name please for the record margot aguilar navarro is navarro  n-a-v-a-r-r-o correct and ms aguilar have you ever been known by any other name other than margot aguilar navarro no so the name  that appears on your birth certificate is this margot aguilar navarro correct have you ever been known by

### Creating\ text files with same name in other directory

In [15]:
import os
#src_dir = '../formatter/all_speech'
tar_dir =  '../formatter/all_text_cleaned'
curr_dir = '/home/nayan/formatter/all_text'

with os.scandir(curr_dir) as entries:
    for entry in entries:
        chk = str(entry)
       
        chk= chk[11:-2]
       
        nam = os.path.join(tar_dir,chk)
        print(nam)
        
       
        #f = open(nam, 'w') 
        
        
      
        
    print('Done!!!')


../formatter/all_text_cleaned/text-form-VS-1332-LX-2021-810359.txt
../formatter/all_text_cleaned/text-form-VS-648-ESQMIA-J7160935.txt
../formatter/all_text_cleaned/text-form-VS-790-ESQMIA-J7283313.txt
../formatter/all_text_cleaned/text-form-VS-804-ESQMIA-J7255074.txt
../formatter/all_text_cleaned/text-form-VS-1287-LX-2021-812995.txt
../formatter/all_text_cleaned/text-form-VS-727-LX-2021-800178.txt
../formatter/all_text_cleaned/text-form-VS-633-ESQMIA-J7151622.txt
../formatter/all_text_cleaned/text-form-VS-848-ESQ-J7271940.txt
../formatter/all_text_cleaned/text-form-VS-838-ESQ-J7292021.txt
../formatter/all_text_cleaned/text-form-VS-1184-ESQ-J7430449.txt
../formatter/all_text_cleaned/text-form-VS-664-ESQMIA-J7162975.txt
../formatter/all_text_cleaned/text-form-VS-774-ESQMIA-J7144499.txt
../formatter/all_text_cleaned/text-form-VS-1299-LX-2021-813219.txt
../formatter/all_text_cleaned/text-form-VS-1075-ESQMIA-J7388804.txt
../formatter/all_text_cleaned/text-form-VS-733-LX-2021-801401.txt
../f

In [3]:
#file= 'all_text/text-form-VS-1035-ESQMIA-J7374393.txt'
#output_file = 'all_text_cleaned/text-form-VS-1035-ESQMIA-J7374393.txt'
#! ./MetalTiger transform -config metaltiger-master/sample_models/lm_clean/en_US/text_cleaning.toml -input $file \
#                    >$output_file

2021/10/25 02:55:04 info  {"MetalTiger Version":"v1.1.0-8945950"}


In [33]:
#! file='all_text/text-form-VS-1035-ESQMIA-J7374393.txt'; outputfile='all_speech/speech.txt'; ./MetalTiger transform -config metaltiger-master/sample_models/lm_clean/en_US/spokenform.toml -input $file \
#     > $outputfile

2021/10/24 07:38:58 info  {"MetalTiger Version":"v1.1.0-8945950"}


In [9]:
#! ./MetalTiger transform -config metaltiger-master/sample_models/lm_clean/en_US/spokenform.toml -input $file> $output_file

/bin/bash: all-speech/speech-form-VS-1166-ESQ-J7385013.txt: No such file or directory


In [31]:
a= '          I         I   S        S       '
b= 'september FIFTEENTH TWO THOUSAND THREE and'
c= 'september ********* *** 15TH     2003'
count =0
for i in range(len(a)):
    #print(i,a[i])
    if a[i]== 'I' or a[i]== 'S':
        if a[i]=='I' and i > 0:
            if a[i-1] != 'I' : 
        count +=1
    
as= ''
at= ''

speech= b[10:]
speech= speech.split(' ')
for i in range(count):
    at= at+ ' '+speech[i]
   
print(at)


 FIFTEENTH TWO THOUSAND THREE


In [35]:
for i in range(0,10,2):
    print(i)

0
2
4
6
8
